A primality test is an algorithm used to determine whether or not a given integer is prime.

The simplest primality test is trial division: $N$ is prime if and only if it is not divisible by any integer $t$ with $1 < t  \leq \sqrt{N}$.

In [2]:
import math

def is_prime(n):
    '''
    A number n is prime if it is not divisible by any integer t with 1 < t <= sqrt(n).
    '''
    if n <= 1:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

def find_primes_in_interval(start, end):
    '''
    Finds all prime numbers in a given interval.
    '''
    return [num for num in range(start, end + 1) if is_prime(num)]

# First interval: [188000, 188200]
start1 = 188000
end1 = 188200
primes1 = find_primes_in_interval(start1, end1)
print(f"Prime numbers in the interval [{start1}, {end1}]:")
print(primes1)

# Second interval: [10^9, 10^9 + 200]
start2 = 10**9
end2 = 10**9 + 200
primes2 = find_primes_in_interval(start2, end2)
print(f"\nPrime numbers in the interval [{start2}, {end2}]:")
print(primes2)

Prime numbers in the interval [188000, 188200]:
[188011, 188017, 188021, 188029, 188107, 188137, 188143, 188147, 188159, 188171, 188179, 188189, 188197]

Prime numbers in the interval [1000000000, 1000000200]:
[1000000007, 1000000009, 1000000021, 1000000033, 1000000087, 1000000093, 1000000097, 1000000103, 1000000123, 1000000181]


Fermat's Little Theorem states that if $p$ is prime then a $p-1 \equiv 1 \mod{p}$ for any $a$ coprime to $p$. The Fermat test base $a$ for $N$, if $1 < a < N$, is to compute
\begin{equation}
    a^{N-1} \mod{N}.
\end{equation}
If this is is not $1$, then $N$ is certainly composite. A Fermat pseudoprime base $a$ is a composite $N$ which passes the Fermat test base $a$.

In [3]:
def modular_pow(base, exponent, modulus):
    '''
    Calculates (base^exponent) % modulus efficiently.
    This function uses the right-to-left binary method.
    '''
    result = 1
    base %= modulus
    while exponent > 0:
        # If exponent is odd, multiply base with the result
        if exponent % 2 == 1:
            result = (result * base) % modulus
        # Exponent must be even now
        exponent = exponent >> 1  # Equivalent to exponent //= 2
        base = (base * base) % modulus
    return result

def fermat_test(N, a):
    '''
    Performs the Fermat primality test for a number N with base a.
    '''
    if modular_pow(a, N - 1, N) != 1:
        return False # N is definitely composite
    return True # N is a probable prime

def run_fermat_on_intervals(start, end, max_a):
    '''
    Runs the Fermat test on an interval of numbers for bases up to max_a.
    Identifies numbers that are likely prime and notes any Fermat pseudoprimes.
    '''
    fermat_pseudoprimes = []
    prime_numbers = []
    for n in range(start, end + 1):
        if n <= 1:
            continue
        is_probable_prime = True
        for a in range(2, max_a + 1):
            if not fermat_test(n, a):
                is_probable_prime = False
                # If it fails for any base, then it's composite
                break
        if is_probable_prime:
            if not is_prime(n):
                fermat_pseudoprimes.append(n)
            else:
                prime_numbers.append(n)
    return fermat_pseudoprimes, prime_numbers

# Define the intervals and the maximum base 'a'
start1, end1 = 188000, 188200
start2, end2 = 10**9, 10**9 + 200
a_limit = 13

# Run the test on the first interval
print(f"Fermat test on the interval [{start1}, {end1}]:")
numbers_1 = run_fermat_on_intervals(start1, end1, a_limit)
print(f"Fermat pseudoprimes: {numbers_1[0]}")
print(f"Prime numbers: {numbers_1[1]}")

# Run the test on the second interval
print(f"\nFermat test on the interval [{start2}, {end2}]:")
numbers_2 = run_fermat_on_intervals(start2, end2, a_limit)
print(f"Fermat pseudoprimes: {numbers_2[0]}")
print(f"Prime numbers: {numbers_2[1]}")

Fermat test on the interval [188000, 188200]:
Fermat pseudoprimes: []
Prime numbers: [188011, 188017, 188021, 188029, 188107, 188137, 188143, 188147, 188159, 188171, 188179, 188189, 188197]

Fermat test on the interval [1000000000, 1000000200]:
Fermat pseudoprimes: []
Prime numbers: [1000000007, 1000000009, 1000000021, 1000000033, 1000000087, 1000000093, 1000000097, 1000000103, 1000000123, 1000000181]


Complexity of trial division:

The main operation inside the loop is the modulo operator, which is a form of division. In the worst-case scenario (when $N$ is prime), the loop runs from $2$ up to $\sqrt{N}$, which means the number of divisions, hence the time complexity is $O(\sqrt{N})$.

Complexity of the Fermat test:

The Fermat test's complexity is determined by the modular exponentiation algorithm, which calculates $a^{N-1} \pmod{N}$. We use an efficient method called binary exponentiation. The main operations inside the loop are at most two multiplications and two modulo divisions. The loop continues halving the exponent as long as the exponent is greater than zero. The number of times you can halve $N-1$ before it becomes zero is approximately $O(\log_2 N)$. Therefore, the time complexity of the Fermat test for $k$ bases is $O(k\log N)$.

Trial division is an exponential time algorithm because the runtime is dependent on the magnitude of $N$, not the number of digits. In contrast, the Fermat Test is a polynomial time algorithm because its runtime is proportional to the number of digits in $N$.

A Carmichael number is a composite number $n$ which satisfies $a^{n-1} \equiv 1 \mod n$ for all integers $a$ that are relatively prime to $n$.

Finding Carmichael numbers by exhaustively checking the Fermat test for many bases is computationally expensive. There are ways to speed up checking for these numbers.

*   If $a$ has a non-trivial common factor with $N$ then we regard $a$ as having determined that $N$ is composite.

*   Korselt's Criterion: A composite number $n$ is a Carmichael number if and only if it is square-free and for every prime factor $p$ of $n$, $(p-1)$ divides $(n-1)$.

*   Carmichael numbers have at least three prime factors.

*   It turns out that the smallest non-Carmichael pseudoprime to prime bases up to $19$ but divisible by $23$ is greater than $10^6$, so we only need to test up to base $19$.

In [4]:
def find_carmichael_numbers(start, end):
    '''
    Carmichael numbers or absolute Fermat pseudoprimes are composite numbers N
    which pass Fermat's test for all bases a coprime to N.
    '''
    carmichael_numbers = []
    for N in range(start, end + 1):
        if is_prime(N):
            continue
        is_carmichael = True
        for a in range(2, 20):
            if math.gcd(a, N) == 1:
                if not fermat_test(N, a):
                    is_carmichael = False
                    break
        if is_carmichael:
            carmichael_numbers.append(N)
    return carmichael_numbers

carmichael_numbers = find_carmichael_numbers(2, 10**6)
print(len(carmichael_numbers))

43


The existence of infinitely many absolute Fermat pseudoprimes means that the Fermat test cannot be relied on to prove the primality of $N$ any faster than trial division, although it can usually detect compositeness quickly.